# DEMO

### Import des librairies et classes nécessaires

In [28]:
from mnist_config import ConfigDict
from mnist_network import MNISTNetwork
from data_provider import MNISTDataset
from text_network import TextLargeMarginModel
from absl import app, flags, logging
import random
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model


In [29]:
MOMENTUM = 0.0
DECAY_STEPS = 2000
DECAY_RATE = 0.96
BATCH_SIZE = 8
LOG_EVERY_STEPS = 50
SAVE_CHECKPOINT_EPOCHS = 5
EXPERIMENT_TYPE = "mnist"
DATA_DIR = "./mnist"   # ou "" pour téléchargement auto

### Chargement des modèles

In [ ]:
model_path_1 = "./models/xent_100epochs_lr0_05_noisy_model.keras"
model_path_2 = "./models/lm_100epochs_lr0_05_noisy_model.keras"
model_path_3 = "./models/xent_100epochs_lr0_01_generalization_batch8_model.keras"
model_path_4 = "./models/lm_100epochs_lr0_01_generalization_batch8_model.keras"
model_path_5 = "./models/text_model_xent_V2_adam.keras"
model_path_6 = "./models/text_model_lm_gamma01_lr0001_adam.keras"
model_path_7 = "./models/text_model_xent_lm_adam.keras"

model_1 = load_model(
    model_path_1,
    compile=False,
    custom_objects={"MNISTNetwork": MNISTNetwork})
model_2 = load_model(
    model_path_2,
    compile=False,
    custom_objects={"MNISTNetwork": MNISTNetwork})
model_3 = load_model(
    model_path_3,
    compile=False,
    custom_objects={"MNISTNetwork": MNISTNetwork})
model_4 = load_model(
    model_path_4,
    compile=False,
    custom_objects={"MNISTNetwork": MNISTNetwork})
model_5 = load_model(
    model_path_5,
    compile=False,
    custom_objects={"TextLargeMarginModel": TextLargeMarginModel})
model_6 = load_model(
    model_path_6,
    compile=False,
    custom_objects={"TextLargeMarginModel": TextLargeMarginModel})
model_7 = load_model(
    model_path_7,
    compile=False,
    custom_objects={"TextLargeMarginModel": TextLargeMarginModel})

model_list_images = [model_1, model_2, model_3, model_4]
model_names_images = ["model_1", "model_2", "model_3", "model_4"]

model_list_texts = [model_5, model_6, model_7]
model_names_texts = ["model_5", "model_6", "model_7

### Récupération du dataset de test MNIST

In [31]:
val_dataset_wrapper = MNISTDataset(
    data_dir=DATA_DIR,
    subset="test",   # ou "test" si c'est MNIST officiel
    batch_size=BATCH_SIZE,
    is_training=False
)

val_dataset = val_dataset_wrapper.dataset
num_examples = val_dataset_wrapper.num_examples
num_classes = val_dataset_wrapper.num_classes

### Résultats de la classification d'image

In [ ]:

for model in model_list_images:
    margin_losses = []
    accuracies = []
    config = ConfigDict()

    for images, labels in val_dataset:
        logits, endpoints = model(images, training=False)

        layers_list = [images]
        for i in range(len(config.filter_sizes_conv_layers)):
            layers_list.append(endpoints[f"conv_layer{i}"])

        one_hot_labels = tf.one_hot(labels, num_classes)

        acc = tf.reduce_mean(tf.cast(tf.argmax(logits, axis=1) == tf.cast(labels, tf.int64), tf.float32))
        accuracies.append(acc.numpy())

    print(f"Validation Accuracy  {model_names_images[model_list_images.index(model)]}:", np.mean(accuracies))

Validation Accuracy  model_1: 0.5032
Validation Accuracy  model_2: 0.8605
Validation Accuracy  model_3: 0.7687
Validation Accuracy  model_4: 0.7527


2026-01-27 19:35:40.449522: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Récupération du dataset de test IMDB

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

VOCAB_SIZE = 10000
NUM_CLASSES = 2
def get_imdb_data():
    print("Chargement IMDB...")
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE)
    
    # On coupe les phrases trop longues (padding)
    x_train = pad_sequences(x_train, maxlen=200, padding='post', truncating='post')
    x_test = pad_sequences(x_test, maxlen=200, padding='post', truncating='post')
    
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(BATCH_SIZE)
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(1000).batch(BATCH_SIZE)
    return train_ds, test_ds

dataset, test_dataset = get_imdb_data()

### Résultats de la classification de texte

In [ ]:
for model in model_list_texts:
    margin_losses = []
    accuracies = []
    config = ConfigDict()

    for images, labels in val_dataset:
        logits, endpoints = model(images, training=False)

        layers_list = [images]
        for i in range(len(config.filter_sizes_conv_layers)):
            layers_list.append(endpoints[f"conv_layer{i}"])

        one_hot_labels = tf.one_hot(labels, num_classes)

        acc = tf.reduce_mean(tf.cast(tf.argmax(logits, axis=1) == tf.cast(labels, tf.int64), tf.float32))
        accuracies.append(acc.numpy())

    print(f"Validation Accuracy  {model_names_texts[model_list_texts.index(model)]}:", np.mean(accuracies))

Validation Accuracy : 0.84204
Validation Accuracy LM: 0.83052
